# Dynamic mode decomposition with control

In this tutorial we show how extend the dynamic mode decomposition to incorporate the effect of control (this technique has been introduced in

> Proctor, Joshua L., Steven L. Brunton, and J. Nathan Kutz. "Dynamic mode decomposition with control." SIAM Journal on Applied Dynamical Systems 15.1 (2016): 142-161. https://doi.org/10.1137/15M1013857

This tutorial originates from the PyDMD package and was adapted for *datafold*

https://github.com/mathLab/PyDMD/blob/master/tutorials/tutorial7/tutorial-7-dmdc.ipynb

We compare and highlight the interface with PyDMD. 

We first import the ``DMDControl`` and ``DMDc`` from the two packages. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy
from pydmd import DMDc

from datafold import DMDControl, TSCDataFrame

We now create a dataset on which we will apply DMD. Since we want add control inputs, the evolution of the complex system can be mathematically described as: 

$$
x_{k+1} = A x_k + B u_k
$$

where the operators $A$ and $B$ are the ones we will approximate using DMD. So, for a illustrative purpose, we create the original snapshots by using two random operators.

In [ ]:
def create_system(n, m):
    A = scipy.linalg.helmert(n, True)
    B = np.random.rand(n, n) - 0.5
    x0 = np.array([0.25] * n)
    u = np.random.rand(n, m - 1) - 0.5
    snapshots = [x0]
    for i in range(m - 1):
        snapshots.append(A.dot(snapshots[i]) + B.dot(u[:, i]))
    snapshots = np.array(snapshots).T
    return {"snapshots": snapshots, "u": u, "B": B, "A": A}

We sample 25 snapshots of the random linear system. For datafold we store the snapshots in the time series collection data structure `TSCDataFrame`.

In [ ]:
s = create_system(25, 10)
print(s["snapshots"].shape)

In [ ]:
X = TSCDataFrame.from_array(s["snapshots"].T)
U = TSCDataFrame.from_array(s["u"].T)

In [ ]:
X.shape

We can now fit the DMD models with the sampled data (both for the PyDMD and datafold version). Note that the control input is a second input. 

In [ ]:
dmdc = DMDc(svd_rank=-1)
dmdc.fit(s["snapshots"], s["u"]);

In [ ]:
dmdcdf = DMDControl()
dmdcdf.fit(X=X, U=U);

Let's compare the original system and the reconstructed system through visualization. We see that the plots from the original system and the two packages are visually identical.

In [ ]:
plt.figure(figsize=(16, 6))

plt.subplot(131)
plt.title("Original system")
plt.pcolor(s["snapshots"].real)
plt.colorbar()

plt.subplot(132)
plt.title("Reconstructed system PyDMD")
plt.pcolor(dmdc.reconstructed_data().real)
plt.colorbar()

plt.subplot(133)
plt.title("Reconstructed system datafold")
plt.pcolor(dmdcdf.reconstruct(X, U=U).to_numpy().T)
plt.colorbar()

We have successfully developed approximations for $A$ and $B$. Finally, the system can now be tested using a new control input. In contrast to previous versions, the desired control input can be provided to the reconstruct methods.

In [ ]:
new_u = np.exp(s["u"])
U_new = TSCDataFrame.from_array(new_u.T)

plt.figure(figsize=(8, 6))

plt.subplot(121)
plt.title("PyDMD")
plt.pcolor(dmdc.reconstructed_data(new_u).real)
plt.colorbar()


plt.subplot(122)
plt.title("datafold")
plt.pcolor(dmdcdf.reconstruct(X=X, U=U_new).T)
plt.colorbar();
